In [28]:
from __future__ import print_function, division, absolute_import
import numpy as np
import os
import re
import h5py
import astropy.units as units
from astropy.time import Time
from pyuvdata import UVData
import hera_pspec
from hera_pspec.data import DATA_PATH

In [20]:
# define days and savepath
days = [str(day) for day in np.arange(2457548, 2457556)]
savepath = '/lustre/aoc/projects/hera/plaplant/test/plots/'

# high band data
hpath = '/lustre/aoc/projects/hera/plaplant/HERA19Golden/CalibratedData/LSThrs_10.5_23.0/FREQrng_530_730/'
hday1 = '2457548_LST_10_23_highband_pI.uvh5.UVP'
hday2 = '2457549_LST_10_23_highband_pI.uvh5.UVP'
hday3 = '2457550_LST_10_23_highband_pI.uvh5.UVP'
hday4 = '2457551_LST_10_23_highband_pI.uvh5.UVP'
hday5 = '2457552_LST_10_23_highband_pI.uvh5.UVP'
hday6 = '2457553_LST_10_23_highband_pI.uvh5.UVP'
hday7 = '2457554_LST_10_23_highband_pI.uvh5.UVP'
hday8 = '2457555_LST_10_23_highband_pI.uvh5.UVP'
hdfiles = [hpath + day for day in [hday1, hday2, hday3, hday4, hday5, hday6, hday7, hday8]]

# low band data
lpath = '/lustre/aoc/projects/hera/plaplant/HERA19Golden/CalibratedData/LSThrs_10.5_23.0/FREQrng_150_350/'
lday1 = '2457548_LST_10_23_lowband_pI.uvh5.UVP'
lday2 = '2457549_LST_10_23_lowband_pI.uvh5.UVP'
lday3 = '2457550_LST_10_23_lowband_pI.uvh5.UVP'
lday4 = '2457551_LST_10_23_lowband_pI.uvh5.UVP'
lday5 = '2457552_LST_10_23_lowband_pI.uvh5.UVP'
lday6 = '2457553_LST_10_23_lowband_pI.uvh5.UVP'
lday7 = '2457554_LST_10_23_lowband_pI.uvh5.UVP'
lday8 = '2457555_LST_10_23_lowband_pI.uvh5.UVP'
ldfiles = [lpath + day for day in [lday1, lday2, lday3, lday4, lday5, lday6, lday7, lday8]]

In [21]:
# read in data
huvps = []
for dfile in hdfiles:
    uvp = hera_pspec.UVPSpec()
    uvp.read_hdf5(dfile)
    huvps.append(uvp)

luvps = []
for dfile in ldfiles:
    uvp = hera_pspec.UVPSpec()
    uvp.read_hdf5(dfile)
    luvps.append(uvp)

In [22]:
# define frequency channels for high band and low band
# full window is 200 channels, but only use central ~100 to account for Blackman-Harris window
# also get corresponding frequencies; assume they are the same for each day
hchans = np.arange(580, 680)
lchans = np.arange(200, 300)
hfreqs = huvps[0].freq_array[np.arange(50, 150)]
lfreqs = luvps[0].freq_array[np.arange(50, 150)]
assert len(hchans) == len(hfreqs)
assert len(lchans) == len(lfreqs)

In [23]:
# compute Jy -> K conversion
cosmo = hera_pspec.conversions.Cosmo_Conversions()
# List of beamfile to load. This is a healpix map.
beamfile = os.path.join(DATA_PATH, 'HERA_NF_efield.beamfits')
# intantiate beam and pass cosmology, if not fed, a default Planck cosmology will be assumed
uvb = hera_pspec.pspecbeam.PSpecBeamUV(beamfile, cosmo=cosmo)
# find conversion factor from Jy to K.  xx and yy are the same.
Jy2K_hb = uvb.Jy_to_mK(hfreqs, pol='xx') / 1000.
Jy2K_lb = uvb.Jy_to_mK(lfreqs, pol='xx') / 1000.

In [7]:
# compute noise estimate for a single day and a single antenna
day = days[0]
ant1 = 9
huvp = huvps[0]
luvp = luvps[0]
caldata_path = os.path.join("/lustre/aoc/projects/hera/plaplant/HERA19Golden/CalibratedData/", day)
caldata = os.listdir(caldata_path)
caldata = sorted([fn for fn in caldata if "xx.HH.uvcRPCS.uvh5" in fn])

# get LST values that went into pspec calc
blps = ((9, 20), (9, 20))
key = (0, blps, 'pI')
spw, bltpairs, pol = huvp.key_to_indices(key)
int_time = huvp.get_integrations(key)
lsts = huvp.lst_avg_array[bltpairs]
valid_lsts = lsts[np.where(int_time > 0)]
valid_int_times = int_time[np.where(int_time > 0)]

# convert int_time from seconds to radians
valid_int_times = valid_int_times * (2 * np.pi) / units.sday.to('s')

# make a range of valid LSTs for high band--each "valid LST" is actually a window of size integration_time around
# that central value
lst_ranges_hb = np.zeros((len(valid_lsts), 2))
for i, lst in enumerate(valid_lsts):
    lst_ranges_hb[i, 0] = lst - valid_int_times[i] / 2
    lst_ranges_hb[i, 1] = lst + valid_int_times[i] / 2
    
# do the same for the low band
spw, bltpairs, pol = luvp.key_to_indices(key)
int_time = luvp.get_integrations(key)
lsts = luvp.lst_avg_array[bltpairs]
valid_lsts = lsts[np.where(int_time > 0)]
valid_int_times = int_time[np.where(int_time > 0)]

# convert int_time from seconds to radians
valid_int_times = valid_int_times * (2 * np.pi) / units.sday.to('s')

lst_ranges_lb = np.zeros((len(valid_lsts), 2))
for i, lst in enumerate(valid_lsts):
    lst_ranges_lb[i, 0] = lst - valid_int_times[i] / 2
    lst_ranges_lb[i, 1] = lst + valid_int_times[i] / 2

# compute Tsys at just the valid LSTs
Tsys_hb = 0.
Tsys_lb = 0.
navg_hb = 0
navg_lb = 0
verbose = False
for fn in caldata:
    uvd_xx = UVData()
    uvd_yy = UVData()
    abspath_xx = os.path.join(caldata_path, fn)
    abspath_yy = re.sub('\.xx\.', '.yy.', abspath_xx)
    if verbose:
        print("Reading {}...".format(abspath_xx))
    uvd_xx.read_uvh5(abspath_xx, ant_str='auto')
    if verbose:
        print("Reading {}...".format(abspath_yy))
    uvd_yy.read_uvh5(abspath_yy, ant_str='auto')
    wf_xx = uvd_xx.get_data(ant1, ant1, 'xx')
    wf_yy = uvd_yy.get_data(ant1, ant1, 'yy')

    file_lsts = np.unique(uvd_xx.lst_array)
    for i, lst in enumerate(file_lsts):
        # do high band first
        if np.any(np.logical_and(lst > lst_ranges_hb[:, 0],
                                 lst < lst_ranges_hb[:, 1])):
            # that lst in the file is in the range of ones used; add its Tsys to the running total
            # pull out just the frequency bins and times we care about
            spec_xx = wf_xx[i, hchans].real * Jy2K_hb
            spec_yy = wf_yy[i, hchans].real * Jy2K_hb
            Tval = ((np.average(spec_xx) + np.average(spec_yy)) / 2)**2
            Tsys_hb += Tval
            navg_hb += 1
        # now low band
        if np.any(np.logical_and(lst > lst_ranges_lb[:, 0],
                                 lst < lst_ranges_lb[:, 1])):
            # that lst in the file is in the range of ones used; add its Tsys to the running total
            # pull out just the frequency bins and times we care about
            spec_xx = wf_xx[i, lchans].real * Jy2K_lb
            spec_yy = wf_yy[i, lchans].real * Jy2K_lb
            Tval = ((np.average(spec_xx) + np.average(spec_yy)) / 2)**2
            Tsys_lb += Tval
            navg_lb += 1

# finish computing averages
if navg_hb > 0:
    Tsys_hb /= navg_hb
else:
    Tsys_hb = 0.
if navg_lb > 0:
    Tsys_lb /= navg_lb
else:
    Tsys_lb = 0.
Tsys_hb = np.sqrt(Tsys_hb)
Tsys_lb = np.sqrt(Tsys_lb)

print("High band Tsys: {:.3f} K".format(Tsys_hb))
print("Low band Tsys:  {:.3f} K".format(Tsys_lb))

High band Tsys: 979.031 K
Low band Tsys:  3031.359 K


In [14]:
# define list of all antennas in UVPspec objects
uvp = huvps[0]
ants = []
for bl in uvp.bl_array:
    antpair = uvp.bl_to_antnums(bl)
    #print("antpair: ", antpair)
    ants.append(antpair[1])
ants = np.unique(ants)
print("ants: ", ants)
key = (0, ((10, 9), (10, 9)), 'pI')
try:
    spw, bltpairs, pol = uvp.key_to_indices(key)
except AssertionError:
    print("not in data")
key = (0, ((9, 10), (9, 10)), 'pI')
try:
    spw, bltpairs, pol = uvp.key_to_indices(key)
except:
    print("nothing here either")
print("bltpairs: ", bltpairs)
print("nonzero blpairs: ", np.count_nonzero(uvp.integration_array[0]))
print("all pairs: ", uvp.integration_array[0].shape)

ants:  [  9  10  20  31  53  64  65  72  88  89  96  97 104 105 112]
not in data
bltpairs:  [128640 128641 128642 ... 130647 130648 130649]
nonzero blpairs:  103052
all pairs:  (211050, 1)


In [49]:
# Compute Tsys for each antenna for each day, as well as array-wide value.
# We use the union of all times that an antenna is used in the pspec
# analysis to define the LST values to include in the noise calculation.
# The "more correct" way would be to use a weighted average of the LSTs
# corresponding to how many bl-pairs that ant was used in for a given LST.
# Assuming that the antenna is used for a similar number of baseline-pairs
# for different LST values, this effect is sub-dominant (and requires a lot
# of bookkeeping to get right).

# initialize Tsys and counters
Nants = len(ants)
Ndays = len(days)
Tsys_hb = np.zeros((Ndays, Nants))
Tsys_lb = np.zeros((Ndays, Nants))
navg_hb = np.zeros((Ndays, Nants), dtype=np.int32)
navg_lb = np.zeros((Ndays, Nants), dtype=np.int32)

for iday, day in enumerate(days):
    huvp = huvps[iday]
    luvp = luvps[iday]
    caldata_path = os.path.join("/lustre/aoc/projects/hera/plaplant/HERA19Golden/CalibratedData/", day)
    caldata = os.listdir(caldata_path)
    caldata = sorted([fn for fn in caldata if "xx.HH.uvcRPCS.uvh5" in fn])

    # get antennas in uvpspec object
    # assume same for low band and high band
    ants = []
    for bl in uvp.bl_array:
        antpair = uvp.bl_to_antnums(bl)
        ants.append(antpair[0])
        ants.append(antpair[1])
    ants = np.unique(ants)

    # get LST values that went into pspec calc
    lsts_hb = []
    lsts_lb = []
    for iant in ants:
        valid_lsts_hb = np.array([])
        valid_int_times_hb = np.array([])
        valid_lsts_lb = np.array([])
        valid_int_times_lb = np.array([])
        for jant in ants:
            # make tuple of blt pair
            if iant < jant:
                bltpair = ((iant, jant), (iant, jant))
            elif iant > jant:
                bltpair = ((jant, iant), (jant, iant))
            else:  # iant == jant
                continue
            key = (0, bltpair, 'pI')
            # high band
            spw, bltpairs, pol = huvp.key_to_indices(key)
            int_time = huvp.get_integrations(key)
            lsts = huvp.lst_avg_array[bltpairs]
            valid_lsts_hb = np.append(valid_lsts_hb, lsts[np.where(int_time > 0)])
            valid_int_times_hb = np.append(valid_int_times_hb, int_time[np.where(int_time > 0)])

            # low band
            spw, bltpairs, pol = luvp.key_to_indices(key)
            int_time = luvp.get_integrations(key)
            lsts = luvp.lst_avg_array[bltpairs]
            valid_lsts_lb = np.append(valid_lsts_lb, lsts[np.where(int_time > 0)])
            valid_int_times_lb = np.append(valid_int_times_lb, int_time[np.where(int_time > 0)])

        # make a unique list now
        # high band
        valid_lsts_hb, inds = np.unique(valid_lsts_hb, return_index=True)
        valid_int_times_hb = valid_int_times_hb[inds]
        # low band
        valid_lsts_lb, inds = np.unique(valid_lsts_lb, return_index=True)
        valid_int_times_lb = valid_int_times_lb[inds]

        # compute LST range
        # high band
        lst_ranges_hb = np.zeros((len(valid_lsts_hb), 2))
        for i, lst in enumerate(valid_lsts_hb):
            lst_ranges_hb[i, 0] = lst - valid_int_times_hb[i] / 2
            lst_ranges_hb[i, 1] = lst + valid_int_times_hb[i] / 2
        # low band
        lst_ranges_lb = np.zeros((len(valid_lsts_lb), 2))
        for i, lst in enumerate(valid_lsts_lb):
            lst_ranges_lb[i, 0] = lst - valid_int_times_lb[i] / 2
            lst_ranges_lb[i, 1] = lst + valid_int_times_lb[i] / 2
            
        # append to running lists
        lsts_hb.append(lst_ranges_hb)
        lsts_lb.append(lst_ranges_lb)

    # compute Tsys at just the valid LSTs
    verbose = False
    for fn in caldata:
        uvd_xx = UVData()
        uvd_yy = UVData()
        abspath_xx = os.path.join(caldata_path, fn)
        abspath_yy = re.sub('\.xx\.', '.yy.', abspath_xx)
        if verbose:
            print("Reading {}...".format(abspath_xx))
        uvd_xx.read_uvh5(abspath_xx, ant_str='auto')
        if verbose:
            print("Reading {}...".format(abspath_yy))
        uvd_yy.read_uvh5(abspath_yy, ant_str='auto')
        file_lsts = np.unique(uvd_xx.lst_array)
        for i, ant in enumerate(ants):
            wf_xx = uvd_xx.get_data(ant, ant, 'xx')
            wf_yy = uvd_yy.get_data(ant, ant, 'yy')

            lst_ranges_hb = lsts_hb[i]
            lst_ranges_lb = lsts_lb[i]
            for j, lst in enumerate(file_lsts):
                # do high band first
                if np.any(np.logical_and(lst > lst_ranges_hb[:, 0],
                                         lst < lst_ranges_hb[:, 1])):
                    # that lst in the file is in the range of ones used; add its Tsys to the running total
                    # pull out just the frequency bins and times we care about
                    spec_xx = wf_xx[j, hchans].real * Jy2K_hb
                    spec_yy = wf_yy[j, hchans].real * Jy2K_hb
                    Tval = ((np.average(spec_xx) + np.average(spec_yy)) / 2)**2
                    Tsys_hb[iday, i] += Tval
                    navg_hb[iday, i] += 1
                # now low band
                if np.any(np.logical_and(lst > lst_ranges_lb[:, 0],
                                         lst < lst_ranges_lb[:, 1])):
                    # that lst in the file is in the range of ones used; add its Tsys to the running total
                    # pull out just the frequency bins and times we care about
                    spec_xx = wf_xx[i, lchans].real * Jy2K_lb
                    spec_yy = wf_yy[i, lchans].real * Jy2K_lb
                    Tval = ((np.average(spec_xx) + np.average(spec_yy)) / 2)**2
                    Tsys_lb[iday, i] += Tval
                    navg_lb[iday, i] += 1

    # finish computing average for that day
    print("\n\nDay {}".format(day))
    for i, ant in enumerate(ants):
        # high band
        if navg_hb[iday, i] > 0:
            Tval_hb = np.sqrt(Tsys_hb[iday, i] / navg_hb[iday, i])
        else:
            Tval_hb = 0.
        # low band
        if navg_lb[iday, i] > 0:
            Tval_lb = np.sqrt(Tsys_lb[iday, i] / navg_lb[iday, i])
        else:
            Tval_lb = 0.
        print("antenna {:d}".format(ant))
        print("Tsys high band: {:.3f} K".format(Tval_hb))
        print("Tsys low band:  {:.3f} K".format(Tval_lb))
    # now compute average
    Tval_hb_avg = np.sqrt(np.sum(Tsys_hb[iday, :]) / np.sum(navg_hb[iday, :]))
    Tval_lb_avg = np.sqrt(np.sum(Tsys_lb[iday, :]) / np.sum(navg_lb[iday, :]))
    print("\nAverage across antennas: ")
    print("Tsys high band: {:.3f} K".format(Tval_hb_avg))
    print("Tsys low band:  {:.3f} K".format(Tval_lb_avg))



Day 2457548
antenna 9
Tsys high band: 842.202 K
Tsys low band:  2897.601 K
antenna 10
Tsys high band: 841.966 K
Tsys low band:  2792.645 K
antenna 20
Tsys high band: 920.002 K
Tsys low band:  3100.750 K
antenna 31
Tsys high band: 919.234 K
Tsys low band:  2794.214 K
antenna 53
Tsys high band: 776.427 K
Tsys low band:  2199.856 K
antenna 64
Tsys high band: 946.873 K
Tsys low band:  3135.891 K
antenna 65
Tsys high band: 1056.210 K
Tsys low band:  3980.007 K
antenna 72
Tsys high band: 1465.987 K
Tsys low band:  7999.601 K
antenna 88
Tsys high band: 1190.050 K
Tsys low band:  3381.887 K
antenna 89
Tsys high band: 1183.769 K
Tsys low band:  4882.016 K
antenna 96
Tsys high band: 1112.649 K
Tsys low band:  3811.026 K
antenna 97
Tsys high band: 878.601 K
Tsys low band:  2788.352 K
antenna 104
Tsys high band: 999.145 K
Tsys low band:  3936.683 K
antenna 105
Tsys high band: 1122.680 K
Tsys low band:  4238.338 K
antenna 112
Tsys high band: 949.556 K
Tsys low band:  3431.286 K

Average across an

In [64]:
# compute and save the Tsys estimate for each antenna for each time (LST) for both bands

# define list of all antennas in UVPspec objects
uvp = huvps[0]
ants = []
for bl in uvp.bl_array:
    antpair = uvp.bl_to_antnums(bl)
    #print("antpair: ", antpair)
    ants.append(antpair[0])
    ants.append(antpair[1])
ants = np.unique(ants)

# loop over days
verbose = False
outdata = "/lustre/aoc/projects/hera/plaplant/test/HERA19/"
for iday, day in enumerate(days):
    print("Processing {}".format(day))

    # scan through the files and figure out the LST grid to use
    caldata_path = os.path.join("/lustre/aoc/projects/hera/plaplant/HERA19Golden/CalibratedData/", day)
    caldata = os.listdir(caldata_path)
    caldata = sorted([fn for fn in caldata if "xx.HH.uvcRPCS.uvh5" in fn])

    lsts = np.array([])
    for fn in caldata:
        uvd = UVData()
        abspath_xx = os.path.join(caldata_path, fn)
        uvd.read_uvh5(abspath_xx, read_data=False)
        lsts = np.append(lsts, np.unique(uvd.lst_array))
    # get the total number of entries
    Nlst = len(lsts)
    Nants = len(ants)

    # make a new array
    # size (Nlst, Nants, Nspw)
    Tsys = np.empty((Nlst, Nants, 2))

    ilst = 0
    # loop through raw data files
    for fn in caldata:
        # read in data
        uvd_xx = UVData()
        uvd_yy = UVData()
        abspath_xx = os.path.join(caldata_path, fn)
        abspath_yy = re.sub('\.xx\.', '.yy.', abspath_xx)
        if verbose:
            print("Reading {}...".format(abspath_xx))
        uvd_xx.read_uvh5(abspath_xx, ant_str='auto')
        if verbose:
            print("Reading {}...".format(abspath_yy))
        uvd_yy.read_uvh5(abspath_yy, ant_str='auto')
        file_lsts = np.unique(uvd_xx.lst_array)
        Nfile_lsts = len(file_lsts)

        for i, ant in enumerate(ants):
            wf_xx = uvd_xx.get_data(ant, ant, 'xx')
            wf_yy = uvd_yy.get_data(ant, ant, 'yy')
            for j, lst in enumerate(file_lsts):
                spec_xx_hb = wf_xx[j, hchans].real * Jy2K_hb
                spec_yy_hb = wf_yy[j, hchans].real * Jy2K_hb
                spec_xx_lb = wf_xx[j, lchans].real * Jy2K_lb
                spec_yy_lb = wf_yy[j, lchans].real * Jy2K_lb
                Tval_hb = (np.average(spec_xx_hb) + np.average(spec_yy_hb)) / 2
                Tval_lb = (np.average(spec_xx_lb) + np.average(spec_yy_lb)) / 2
                idx = ilst + j
                
                # add to proper index in array
                Tsys[idx, i, 0] = Tval_hb
                Tsys[idx, i, 1] = Tval_lb
                
        ilst += Nfile_lsts
        
    # make sure we ended up at the right spot
    assert ilst == Nlst
    
    # save out file
    fn_out = os.path.join(outdata, "Tsys_{}.h5".format(day))
    if os.path.isfile(fn_out):
        os.remove(fn_out)
    print("saving {}".format(fn_out))
    t = Time.now()
    with h5py.File(fn_out, 'w') as f:
        header = f.create_group("Header")
        header["Nlst"] = Nlst
        header["Nants"] = Nants
        header["history"] = np.string_("Generated at {}".format(t.iso))
        data = f.create_group("Data")
        data["lst_array"] = lsts
        data["ants"] = ants
        data["Tsys"] = Tsys

Processing 2457548
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457548.h5
Processing 2457549
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457549.h5
Processing 2457550
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457550.h5
Processing 2457551
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457551.h5
Processing 2457552
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457552.h5
Processing 2457553
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457553.h5
Processing 2457554
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457554.h5
Processing 2457555
saving /lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457555.h5


In [ ]:
# compute Tsys based on baselines in pspec objects
#
# We assume that for a given baseline (i, j), the "Tsys" value is:
#    T_{i,j} = sqrt(T_i * T_j),
# where
#    T_i = <V_{i,i}(nu) * B(nu)>_nu,
# that is, the average value over frequency of the (calibrated) auto-correlation
# multiplied by the conversion factor to go from Jy -> K. Assuming the T_i
# values from different antennas are independent (which is almost certainly wrong),
# then the combined temperature value T_{i,j} is just the harmonic mean of their
# individual values.
#
# In power-spectrum space, we form power spectra from cross-multiplying two different
# spectra. Their noise is characterized by a single system temperature Tsys, i.e.,
#     P_n(k) ~ Tsys^2.
# Because P(k) is formed from visibilities (i,j) and (k,l), we can express this as:
#     P_n(k) ~ T_{i,j}T_{k,l}
#  =>   Tsys = sqrt(T_{i,j}T_{k,l})
#            = sqrt(sqrt(T_i*T_j)*sqrt(T_k*T_l))
#            = (T_i*T_j*T_k*T_l)^(1/4)
# This is the Tsys value for a single baseline and a single LST value. To compute the
# weighted value over all times, we combine the individual Tsys estimates in quadrature.

In [65]:
# define where Tsys data lives
outdata = "/lustre/aoc/projects/hera/plaplant/test/HERA19/"

# loop over pspec objects for each day
for iday, day in enumerate(days):
    print("Day: ", day)
    # get uvpspec objects
    huvp = huvps[iday]
    luvp = luvps[iday]

    # get Tsys values
    Tsys_file = os.path.join(outdata, "Tsys_{}.h5".format(day))
    with h5py.File(Tsys_file, "r") as f:
        lst_array = f["Data/lst_array"].value
        ants = f["Data/ants"].value
        Tsys = f["Data/Tsys"].value
    # Initialize Tsys and counters
    Tsys_hb = 0.
    Tsys_lb = 0.
    navg_hb = 0
    navg_lb = 0
    
    # Loop over all bl-pairs in pspec objects
    # high band first
    uvp = huvps[iday]
    blps = uvp.get_blpairs()
    for blp in blps:
        ((ant1, ant2), (ant3, ant4)) = blp
        iant = np.argwhere(ants == ant1)[0][0]
        jant = np.argwhere(ants == ant2)[0][0]
        kant = np.argwhere(ants == ant3)[0][0]
        lant = np.argwhere(ants == ant4)[0][0]
        key = (0, blp, 'pI')
        spw, inds, pol = uvp.key_to_indices(key)
        int_time = uvp.integration_array[spw][inds, pol]
        lsts = uvp.lst_avg_array[inds]
        for itime, lst in zip(int_time, lsts):
            if itime == 0:
                continue
            idx = (np.abs(lst_array - lst).argmin())
            # compute T and add to total
            Ti = Tsys[idx, iant, 0]
            Tj = Tsys[idx, jant, 0]
            Tk = Tsys[idx, kant, 0]
            Tl = Tsys[idx, lant, 0]
            T = np.sqrt(Ti * Tj * Tk * Tl)
            Tsys_hb += T
            navg_hb += 1

    # low band
    uvp = luvps[iday]
    blps = uvp.get_blpairs()
    for blp in blps:
        ((ant1, ant2), (ant3, ant4)) = blp
        iant = np.argwhere(ants == ant1)[0][0]
        jant = np.argwhere(ants == ant2)[0][0]
        kant = np.argwhere(ants == ant3)[0][0]
        lant = np.argwhere(ants == ant4)[0][0]
        key = (0, blp, 'pI')
        spw, inds, pol = uvp.key_to_indices(key)
        int_time = uvp.integration_array[spw][inds, pol]
        lsts = uvp.lst_avg_array[inds]
        for itime, lst in zip(int_time, lsts):
            if itime == 0:
                continue
            idx = (np.abs(lst_array - lst).argmin())
            # compute T and add to total
            Ti = Tsys[idx, iant, 1]
            Tj = Tsys[idx, jant, 1]
            Tk = Tsys[idx, kant, 1]
            Tl = Tsys[idx, lant, 1]
            T = np.sqrt(Ti * Tj * Tk * Tl)
            Tsys_lb += T
            navg_lb += 1

    # Finish computing averages
    Tsys_hb /= navg_hb
    Tsys_hb = np.sqrt(Tsys_hb)
    Tsys_lb /= navg_lb
    Tsys_lb = np.sqrt(Tsys_lb)
    print("Tsys, high band: {:4.3f} K".format(Tsys_hb))
    print("Tsys, low band:  {:4.3f} K".format(Tsys_lb))

Day:  2457548
Tsys, high band: 1167.663 K
Tsys, low band:  3867.379 K
Day:  2457549
Tsys, high band: 1218.223 K
Tsys, low band:  4074.381 K
Day:  2457550
Tsys, high band: 1258.792 K
Tsys, low band:  4022.913 K
Day:  2457551
Tsys, high band: 1282.114 K
Tsys, low band:  3881.541 K
Day:  2457552
Tsys, high band: 1239.054 K
Tsys, low band:  4007.365 K
Day:  2457553
Tsys, high band: 1224.988 K
Tsys, low band:  3902.342 K
Day:  2457554
Tsys, high band: 1164.618 K
Tsys, low band:  3982.286 K
Day:  2457555
Tsys, high band: 1263.639 K
Tsys, low band:  3907.681 K


In [56]:
f = h5py.File('/lustre/aoc/projects/hera/plaplant/test/HERA19/Tsys_2457548.h5', 'r')
f['Data/Tsys']

<HDF5 dataset "Tsys": shape (4021, 15, 2), type "<f8">